# CGEM the Notebook

This Notebook gives an overview of looking at CGEM output with Python.

## Compile the code

In [ ]:
!make

## Import CGEM Python functions
Current library of functions are in **cgem.py**, absolutely devoid of commenting and error checking.  (Sorry, try back later.)

Click the folder icon on the left to see the files.

In [ ]:
#Imports all the Python functions from cgem.nml and keep the original names
from cgem import *

---

## Basics of namelists

CGEM reads the input parameters from namelists:
- **grid.nml**: parameters that should come from the hydro code
- **cgem.nml**: parameters that should (mostly) be private to the biogeochem code

Use **f90nml** to read and manipulate namelists.  See:
- https://f90nml.readthedocs.io/en/latest/index.html

In [ ]:
#Import f90nml
import f90nml

### The grid parameters:

In [ ]:
#Read and print grid parameters
grid = f90nml.read('grid.nml')
print("Grid parameters")
print(grid)

### The CGEM parameters

In [ ]:
#Read and print cgem parameters
cgem = f90nml.read('cgem.nml')
print("CGEM parameters")
print(cgem)

### Get the value of a single element
```
value = namelist_object.get('named_list').get('parameter')
```

In [ ]:
## Get the number of layers and number of phytoplankton species
km = grid.get('hydro').get('km')
nospA = cgem.get('nosp').get('nospa')
print("There are ",km," layers and ",nospA," phytoplankton species")

---

## The CGEM Python library

See **cgem.py** for the details.

### Functions:
- `cgem_plot1D(grid,var)`: make timeseries plot of state variable `var` at layer `k=1`
- `cgem_plotks(grid,var)`: make timeseries plot of a state variable `var` at each layer `k`
- `cgem_plotAs(grid,cgem)`: make timeseries plot of each phytoplankton group at layer `k=1`
- `cgem_getvar(var)`: gets a state variable `var` at layer `k=1`. 
- `cgem_tstart(grid)`: gets datetime object for start of simulation
- `cgem_tend(grid)`:  gets datetime object for end of simulation
- `cgem_timearray(grid)`: A time array, defined by the beginning of simulation and the array length of results from a run, rather than simulation start and end times.  (This is in case the code crashed.) 

### To work with the output, save to a variable
For example:
```
phytoplankton = cgem_getvar('A')
```
or
```
time = cgem_timearray(grid)
```

### Once you save the variables, you can calculate stuff or make your own plots

In [ ]:
A = cgem_getvar("A")
print(A)
print("mean, max, min of A:",A.mean(),max(A),min(A))

## Basic plots

This always plots layer `k=1`.

syntax:
```
cgem_plot1D(grid,var)
```

Options for var are:
```
A, Qn, Qp, Z, NO3, NH4, PO4, DIC, O2, OM1_A, OM2_A, OM1_Z, OM2_Z, OM1_R, OM2_R, CDOM, Si, Alk, Tr
```

If using more than one species, use the name and number, e.g., `A3`.

In [ ]:
#Plot the first phytoplankton group, use 'A' or 'A1'
cgem_plot1D(grid,'A')

---

## Modifying the namelists
You can directly edit the **.nml** files by clicking on them, editing, and saving, but it is actually more reliable to do this in Python. The Fortran code won't notice if you don't define the correct number of parameters and will crash with a segmentation fault if you do that wrong.

Some notes:
- Fortran isn't case sensitive, but Python is.  Use all lower case when using f90nml.
- Using `force=True` with nml.write will overwrite the nml file.  The CGEM code needs those files to be named **grid.nml** and **cgem.nml**.
- If you mess something up and want to start with the default **cgem** or **grid.nml**, do `!cp nml_save/cgem.nml .` and `!cp nml_save/grid.nml .`  
- You can check the difference between current and default nmls by doing `!diff grid.nml nml_save/grid.nml`.  
- In Python, you can do `![whatever the regular shell command is]`.

### Example: Change the simulation length

In [ ]:
#Check end of simulation month
print(grid['hydro']['imone'])
#Change it to March
grid['hydro']['imone'] = 3
# Check it
print(grid['hydro']['imone'])
# Write the nml file
grid.write('grid.nml',force=True)
#Redo the phytoplankton plot
cgem_plot1D(grid,'A')

### Resetting the nml file
Use the `cp` command from the shell, then redefine the namelist object.

In [ ]:
!cp nml_save/grid.nml .
grid = f90nml.read('grid.nml')

In [ ]:
#Redo the phytoplankton plot
cgem_plot1D(grid,'A2')

---

## Making custom plots

So far, there is a limited selection of canned plot functions.  When making plots, define a time array if you don't want the x-axis to be 'timesteps'.

In [ ]:
T = cgem_timearray(grid)

In [ ]:
plt.plot(T,A)

## That's all we got for now.
To plot a bunch of variables on the same plot, or change colors and stuff, you write your own Python...

In [ ]:
Z = cgem_getvar('Z')
NO3 = cgem_getvar('NO3')
NH4 = cgem_getvar('NH4')
PO4 = cgem_getvar('PO4')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(T,NO3)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(T,Z)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(T,NO3, color='black')
ax.plot(T,NH4, color='blue')
ax.plot(T,PO4, color='cyan')